In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from pymongo import MongoClient

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Imports the Pythod CRUD module
from MongoAACCrud import AnimalShelter

###########################
# Data Manipulation / Model
###########################

#USER and PASS handled in the default constructor of AnimalShelter()
shelter = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
try:
    df = pd.DataFrame.from_records(shelter.read({}))
except Exception as e:
    print(f"Error: {e}")

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div(style={'backgroundColor': '#D2F3FF'}, children=[
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('Hunter Richards CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'height': '10%', 'width': '10%', 'display': 'block', 'margin': 'auto'}
        ),
        style={'text-align': 'center'}
    ),
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WATER RESCUE'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'MOUNTAIN OR WILDERNESS RESCUE'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'DISASTER OR INDIVIDUAL TRACKING'},
                {'label': 'Reset', 'value': 'RESET'}
            ]
        )
    ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size = 10,
        selected_rows = [],
        selected_columns = [],
        sort_action = 'native',
        sort_mode = 'multi',
        editable = False,
        column_selectable = False,
        page_current = 0,
        row_deletable = False,
        page_action = 'native',
        filter_action = 'native',
        row_selectable = 'single'
    ),
    html.Br(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div([
                     dcc.Dropdown(
                         id='piechartdropdown-id',
                         className='col s12 m6',
                         options=[
                             {'label': 'Breed', 'value': 'breed'},
                             {'label': 'Age', 'value': 'age_upon_outcome'},
                             {'label': 'Outcome Type', 'value': 'outcome_type'}
                         ],
                         value='breed',
                         multi=False,
                         clearable=False
                     )
                 ],
                     style={"width": "5%"}
                 ),
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                     style={"width": "30%"}
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 ),
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################
# Callback to update data, columns, and selected rows of data table
@app.callback([Output('datatable-id','data'),
            Output('datatable-id','columns'),
            Output('datatable-id','selected_rows')],
            [Input('filter-type', 'value')])

# Dashboard update based on filter type
def update_dashboard(filter_type):
    
    # If RESET, read all documents in the data table
    if filter_type == 'RESET':
        df = pd.DataFrame.from_records(shelter.read({}))
        
    # If WATER, read only specific documents based on the following:
    elif filter_type == 'WATER RESCUE':
        df = pd.DataFrame(list(shelter.read({
            "animal_type":"Dog",
            "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome":"Intact Female",
            "age_upon_outcome_in_weeks":{"$gte": 26},
            "age_upon_outcome_in_weeks":{"$lte": 156},
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
    # If MOUNTAIN OR WILDERNESS, read only specific documents based on the following:
    elif filter_type == 'MOUNTAIN OR WILDERNESS RESCUE':
        df = pd.DataFrame(list(shelter.read({
            "animal_type":"Dog",
            "breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte": 26},
            "age_upon_outcome_in_weeks":{"$lte": 156},
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
    # If DISASTER OR INDIVIDUAL TRACKING, read only specific documents based on the following:
    elif filter_type == 'DISASTER OR INDIVIDUAL TRACKING':
        df = pd.DataFrame(list(shelter.read({
            "animal_type":"Dog",
            "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome":"Intact Male",
            "age_upon_outcome_in_weeks":{"$gte": 20},
            "age_upon_outcome_in_weeks":{"$lte": 300},
            "outcome_type":{"$nin":["Euthanasia"]}
        })))
    #Else, read all documents
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
        
    # Prepares data and columns sent to data table
    data=df.to_dict('records')
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    # Resets selected row to first row
    selected_rows = [0]
    
    return (data, columns, selected_rows)


# Callback function to update graph, data table, and pie-chart based on selected data from data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('piechartdropdown-id', 'value')])
def update_graphs(viewData, dropdownValue):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, names=dropdownValue)
        )    
    ]
# Callback function to update geolocation map based on selected row from data table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

#################
# Geolocation Map
#################

def update_map(viewData, selectedRows):   
    
    # Convert the view data to a dataframe for easier processing
    dff = pd.DataFrame.from_dict(viewData)
    
    # If no row is selected, default to the first row, otherwise use the selected row
    selected_row = 0 if not selectedRows else selectedRows[0]   

    # Extract the x and y coordinates for the selected row
    posX, posY = dff.iloc[selected_row, 13], dff.iloc[selected_row, 14]
    
    # Ensure that posX and posY are valid numbers
    if not np.isfinite(posX) or not np.isfinite(posY):
        return html.Div("Invalid or missing geo-coordinates!")

    # Get the name associated with the selected row, handle missing names
    name = dff.iloc[selected_row, 9] 
    name = name if name else "-Name Missing-"

    # Construct and return a geolocation map with relevant markers and information
    return [
        dl.Map(style={'width': '1600px', 'height': '450px'}, center=[posX, posY], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[posX, posY], children=[
                dl.Tooltip(dff.iloc[selected_row, 4]),
                dl.Popup([
                    html.P("Animal Name"),
                    html.H1(name)
                ])
            ])
        ])
    ]   



app.run_server(debug=True)


Dash app running on http://127.0.0.1:9767/
